In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
import warnings
# 경고창 숨기기
warnings.filterwarnings('ignore') 
raw = pd.read_csv('all_merge.csv')
raw = raw.iloc[:,1:]
raw.replace([np.inf, -np.inf], 0, inplace=True)
raw['Loan_Apply'] = raw['ViewLoanApplyIntro']+raw['StartLoanApply']+raw['CompleteIDCertification']+raw['EndLoanApply']

In [2]:
col = ['danger_score','bank_id','loan_limit','loan_rate','desired_amount','purpose','is_applied']
raw_col = raw[col]
raw_col['Diff'] = 0
raw_col['Diff'] = np.abs(raw_col['desired_amount'] + 1 - raw_col['loan_limit'])/(raw_col['desired_amount']+1)
raw_col['dd'] = raw_col['desired_amount'] - raw_col['loan_limit']
raw_col.loc[raw_col['dd'] <= 0, 'Diff'] = 0
del raw_col['dd']
raw['Diff'] = raw_col['Diff']

In [3]:
del raw['loanapply_insert_time']
del raw['date_cd']
del raw['rate_old']
del raw['rate_new']
del raw['desired_amount']
del raw['ViewLoanApplyIntro']
del raw['StartLoanApply']
del raw['EndLoanApply']
del raw['CompleteIDCertification']
del raw['GetCreditInfo']
del raw['Login']
del raw['OpenApp']
del raw['SignUp']
del raw['UseDSRCalc']
del raw['UseLoanManage']
del raw['UsePrepayCalc']
del raw['Loan_Apply']
del raw['houseown_type']
del raw['insert_time']

In [4]:
raw.columns

Index(['application_id', 'bank_id', 'product_id', 'loan_limit', 'loan_rate',
       'is_applied', 'user_id', 'credit_score', 'yearly_income', 'income_type',
       'employment_type', 'purpose', 'existing_loan_cnt', 'existing_loan_amt',
       'personal_rehabilitation', 'working', 'DTI', 'danger_score', 'Diff'],
      dtype='object')

In [5]:
raw

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,credit_score,yearly_income,income_type,employment_type,purpose,existing_loan_cnt,existing_loan_amt,personal_rehabilitation,working,DTI,danger_score,Diff
0,566528,13,123,20000000.0,19.1,0.0,681184,580.0,8000000.0,OTHERINCOME,기타,생활비,4.0,20000000.0,2.0,1.0,0.611225,92.976939,0.00
1,180433,19,231,16000000.0,15.0,0.0,623737,740.0,12000000.0,FREELANCER,일용직,생활비,3.0,11000000.0,2.0,0.0,0.273075,71.488470,0.00
2,180433,13,262,22000000.0,16.6,0.0,623737,740.0,12000000.0,FREELANCER,일용직,생활비,3.0,11000000.0,2.0,0.0,0.273075,79.874214,0.00
3,180433,21,196,3000000.0,10.9,1.0,623737,740.0,12000000.0,FREELANCER,일용직,생활비,3.0,11000000.0,2.0,0.0,0.273075,50.000000,0.00
4,180433,1,61,3000000.0,14.9,0.0,623737,740.0,12000000.0,FREELANCER,일용직,생활비,3.0,11000000.0,2.0,0.0,0.273075,70.964361,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12715887,634459,10,65,30000000.0,14.5,NaN,244440,800.0,30000000.0,PRACTITIONER,정규직,생활비,1.0,8000000.0,2.0,0.0,0.103103,67.557652,0.00
12715888,634459,19,231,11000000.0,16.6,NaN,244440,800.0,30000000.0,PRACTITIONER,정규직,생활비,1.0,8000000.0,2.0,0.0,0.103103,78.563941,0.00
12715889,634459,10,149,30000000.0,14.5,NaN,244440,800.0,30000000.0,PRACTITIONER,정규직,생활비,1.0,8000000.0,2.0,0.0,0.103103,67.557652,0.00
12715890,1288711,35,267,3000000.0,13.8,NaN,450880,750.0,45000000.0,PRACTITIONER,정규직,대환대출,0.0,0.0,2.0,21.0,0.230111,63.888889,0.94


# 생활비

In [6]:
purpose=raw[(raw['purpose']=='생활비') | (raw['purpose'] == '기타')]
del purpose['purpose']
purpose_ = purpose.drop(labels=['application_id','user_id','loan_rate','loan_limit'],axis=1)
purpose_=pd.get_dummies(purpose_, columns = ['income_type', 'employment_type'])
purpose_['yearly_income'] = np.log1p(purpose_['yearly_income'])
purpose_['existing_loan_amt'] = np.log1p(purpose_['existing_loan_amt'])
purpose_trn=purpose_[purpose_['is_applied'].notnull()]
purpose_test=purpose_[purpose_['is_applied'].isnull()]
y=purpose_trn['is_applied']
X=purpose_trn.drop(['is_applied'], axis=1)
from sklearn.model_selection import train_test_split
purpose_trn_x,purpose_val_x,purpose_trn_y,purpose_val_y=train_test_split(X,y)

In [7]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(purpose_trn_x, purpose_trn_y)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
rf_living = RandomForestClassifier(max_depth = 20, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 300)
rf_living.fit(X_resampled, y_resampled)

rfCpred = rf_living.predict(purpose_val_x)
print(f1_score(purpose_val_y,rfCpred))

0.3083568178497852


# 대환대출

In [9]:
purpose=raw[(raw['purpose']=='대환대출')]
del purpose['purpose']
purpose_ = purpose.drop(labels=['application_id','user_id','loan_rate','loan_limit'],axis=1)
purpose_=pd.get_dummies(purpose_, columns = ['income_type', 'employment_type'])
purpose_['yearly_income'] = np.log1p(purpose_['yearly_income'])
purpose_['existing_loan_amt'] = np.log1p(purpose_['existing_loan_amt'])
purpose_trn=purpose_[purpose_['is_applied'].notnull()]
purpose_test=purpose_[purpose_['is_applied'].isnull()]
y=purpose_trn['is_applied']
X=purpose_trn.drop(['is_applied'], axis=1)
from sklearn.model_selection import train_test_split
purpose_trn_x,purpose_val_x,purpose_trn_y,purpose_val_y=train_test_split(X,y)

In [10]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
purpose_trn_over_x,purpose_trn_over_y = smote.fit_resample(purpose_trn_x,purpose_trn_y)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
rf_switch = RandomForestClassifier(max_depth = 20, min_samples_leaf = 2, min_samples_split = 2, n_estimators = 200,n_jobs=-1)
rf_switch.fit(purpose_trn_over_x,purpose_trn_over_y)

rfCpred = rf_switch.predict(purpose_val_x)
print(f1_score(purpose_val_y,rfCpred))

0.3016311657822015


# 투자

In [12]:
purpose=raw[(raw['purpose']=='투자')]
del purpose['purpose']
purpose_ = purpose.drop(labels=['application_id','user_id','loan_rate','loan_limit'],axis=1)
purpose_=pd.get_dummies(purpose_, columns = ['income_type', 'employment_type'])
purpose_['yearly_income'] = np.log1p(purpose_['yearly_income'])
purpose_['existing_loan_amt'] = np.log1p(purpose_['existing_loan_amt'])
purpose_trn=purpose_[purpose_['is_applied'].notnull()]
purpose_test=purpose_[purpose_['is_applied'].isnull()]
y=purpose_trn['is_applied']
X=purpose_trn.drop(['is_applied'], axis=1)
from sklearn.model_selection import train_test_split
purpose_trn_x,purpose_val_x,purpose_trn_y,purpose_val_y=train_test_split(X,y)

In [13]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
purpose_trn_over_x,purpose_trn_over_y = smote.fit_resample(purpose_trn_x,purpose_trn_y)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
rf_invest= RandomForestClassifier(max_depth = 20, min_samples_leaf = 5, min_samples_split = 2, n_estimators = 200,n_jobs=-1)
rf_invest.fit(purpose_trn_over_x,purpose_trn_over_y)

rfCpred = rf_invest.predict(purpose_val_x)
print(f1_score(purpose_val_y,rfCpred))

0.2756598240469208


# 사업자금

In [15]:
purpose=raw[(raw['purpose']=='사업자금')]
del purpose['purpose']
purpose_ = purpose.drop(labels=['application_id','user_id','loan_rate','loan_limit'],axis=1)
purpose_=pd.get_dummies(purpose_, columns = ['income_type', 'employment_type'])
purpose_['yearly_income'] = np.log1p(purpose_['yearly_income'])
purpose_['existing_loan_amt'] = np.log1p(purpose_['existing_loan_amt'])
purpose_trn=purpose_[purpose_['is_applied'].notnull()]
purpose_test=purpose_[purpose_['is_applied'].isnull()]
y=purpose_trn['is_applied']
X=purpose_trn.drop(['is_applied'], axis=1)
from sklearn.model_selection import train_test_split
purpose_trn_x,purpose_val_x,purpose_trn_y,purpose_val_y=train_test_split(X,y)

In [16]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
purpose_trn_over_x,purpose_trn_over_y = smote.fit_resample(purpose_trn_x,purpose_trn_y)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
rf_business= RandomForestClassifier(max_depth = 20, min_samples_leaf = 5, min_samples_split = 2, n_estimators = 200,n_jobs=-1)
rf_business.fit(purpose_trn_over_x,purpose_trn_over_y)

rfCpred = rf_business.predict(purpose_val_x)
print(f1_score(purpose_val_y,rfCpred))

0.3703357482277791


In [18]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(purpose_val_y,rfCpred)
print(cf)

[[93754  4026]
 [ 4057  2377]]


# 자동차구입

In [19]:
purpose=raw[(raw['purpose']=='자동차구입')]
del purpose['purpose']
purpose_ = purpose.drop(labels=['application_id','user_id','loan_rate','loan_limit'],axis=1)
purpose_=pd.get_dummies(purpose_, columns = ['income_type', 'employment_type'])
purpose_['yearly_income'] = np.log1p(purpose_['yearly_income'])
purpose_['existing_loan_amt'] = np.log1p(purpose_['existing_loan_amt'])
purpose_trn=purpose_[purpose_['is_applied'].notnull()]
purpose_test=purpose_[purpose_['is_applied'].isnull()]
y=purpose_trn['is_applied']
X=purpose_trn.drop(['is_applied'], axis=1)
from sklearn.model_selection import train_test_split
purpose_trn_x,purpose_val_x,purpose_trn_y,purpose_val_y=train_test_split(X,y)

In [20]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
purpose_trn_over_x,purpose_trn_over_y = smote.fit_resample(purpose_trn_x,purpose_trn_y)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
rf_car= RandomForestClassifier(max_depth = 30, min_samples_leaf = 10, min_samples_split = 10, n_estimators = 300,n_jobs=-1)
rf_car.fit(purpose_trn_over_x,purpose_trn_over_y)

rfCpred = rf_car.predict(purpose_val_x)
print(f1_score(purpose_val_y,rfCpred))

0.3112480739599384


# 전월세 보증금

In [22]:
purpose=raw[(raw['purpose']=='전월세보증금')]
del purpose['purpose']
purpose_ = purpose.drop(labels=['application_id','user_id','loan_rate','loan_limit'],axis=1)
purpose_=pd.get_dummies(purpose_, columns = ['income_type', 'employment_type'])
purpose_['yearly_income'] = np.log1p(purpose_['yearly_income'])
purpose_['existing_loan_amt'] = np.log1p(purpose_['existing_loan_amt'])
purpose_trn=purpose_[purpose_['is_applied'].notnull()]
purpose_test=purpose_[purpose_['is_applied'].isnull()]
y=purpose_trn['is_applied']
X=purpose_trn.drop(['is_applied'], axis=1)
from sklearn.model_selection import train_test_split
purpose_trn_x,purpose_val_x,purpose_trn_y,purpose_val_y=train_test_split(X,y)

In [23]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
purpose_trn_over_x,purpose_trn_over_y = smote.fit_resample(purpose_trn_x,purpose_trn_y)

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
rf_home= RandomForestClassifier(max_depth = 20, min_samples_leaf = 3, min_samples_split = 2, n_estimators = 200,n_jobs=-1)
rf_home.fit(purpose_trn_over_x,purpose_trn_over_y)

rfCpred = rf_home.predict(purpose_val_x)
print(f1_score(purpose_val_y,rfCpred))

0.2871794871794872


# 주택구입

In [25]:
purpose=raw[(raw['purpose']=='주택구입')]
del purpose['purpose']
purpose_ = purpose.drop(labels=['application_id','user_id','loan_rate','loan_limit'],axis=1)
purpose_=pd.get_dummies(purpose_, columns = ['income_type', 'employment_type'])
purpose_['yearly_income'] = np.log1p(purpose_['yearly_income'])
purpose_['existing_loan_amt'] = np.log1p(purpose_['existing_loan_amt'])
purpose_trn=purpose_[purpose_['is_applied'].notnull()]
purpose_test=purpose_[purpose_['is_applied'].isnull()]
y=purpose_trn['is_applied']
X=purpose_trn.drop(['is_applied'], axis=1)
from sklearn.model_selection import train_test_split
purpose_trn_x,purpose_val_x,purpose_trn_y,purpose_val_y=train_test_split(X,y)

In [26]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374378 entries, 990 to 9459949
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   bank_id                      374378 non-null  int64  
 1   product_id                   374378 non-null  int64  
 2   credit_score                 374378 non-null  float64
 3   yearly_income                374378 non-null  float64
 4   existing_loan_cnt            374378 non-null  float64
 5   existing_loan_amt            374378 non-null  float64
 6   personal_rehabilitation      374378 non-null  float64
 7   working                      374378 non-null  float64
 8   DTI                          374378 non-null  float64
 9   danger_score                 374378 non-null  float64
 10  Diff                         374378 non-null  float64
 11  income_type_EARNEDINCOME     374378 non-null  uint8  
 12  income_type_EARNEDINCOME2    374378 non-null  uint8  
 

In [27]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
purpose_trn_over_x,purpose_trn_over_y = smote.fit_resample(purpose_trn_x,purpose_trn_y)

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
rf_home1= RandomForestClassifier(max_depth = 30, min_samples_leaf = 12, min_samples_split = 10, n_estimators = 300,n_jobs=-1)
rf_home1.fit(purpose_trn_over_x,purpose_trn_over_y)

rfCpred = rf_home1.predict(purpose_val_x)
print(f1_score(purpose_val_y,rfCpred))

0.2343883661248931


# 결과 예측

In [29]:
test=raw[raw['is_applied'].isnull()]
prediction=pd.DataFrame(columns=['product_id','application_id','is_applied','loan_limit','loan_rate'])

In [30]:
test.replace('기타','생활비',inplace=True)

pur_list=test['purpose'].unique()

model_list=[rf_living,rf_switch,rf_invest,rf_home1,rf_home,rf_invest,rf_car]

test=pd.get_dummies(test, columns = ['employment_type','income_type'])
 

for i in range(len(pur_list)):

    data=test[test['purpose']==pur_list[i]]
    rate = data['loan_rate']
    limit = data['loan_limit']

    del data['purpose']
    del data['loan_rate']
    del data['loan_limit']
    
    data=pd.get_dummies(data)

    pro_id=data['product_id']

    app_id=data['application_id']

    data=data.drop(labels=['application_id','user_id','is_applied'],axis=1)

    pred=model_list[i].predict(data)

    new=pd.DataFrame(columns=['product_id','application_id','is_applied'])

    new['product_id']=pro_id

    new['application_id']=app_id
    
    new['loan_limit'] = limit
    
    new['loan_rate'] = rate

    new['is_applied']=pred

    prediction=prediction.append(new)


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2812 entries, 9463300 to 12704384
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   bank_id                      2812 non-null   int64  
 1   product_id                   2812 non-null   int64  
 2   credit_score                 2812 non-null   float64
 3   yearly_income                2812 non-null   float64
 4   existing_loan_cnt            2812 non-null   float64
 5   existing_loan_amt            2812 non-null   float64
 6   personal_rehabilitation      2812 non-null   float64
 7   working                      2812 non-null   float64
 8   DTI                          2812 non-null   float64
 9   danger_score                 2812 non-null   float64
 10  Diff                         2812 non-null   float64
 11  employment_type_계약직          2812 non-null   uint8  
 12  employment_type_생활비          2812 non-null   uint8  
 13  employme

In [32]:
loan = pd.read_csv('loan_result.csv')
re = loan[loan['is_applied'].isnull()]
re = re[~re.duplicated(keep='first')]
re

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN
...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN


In [33]:
a = pd.merge(re, prediction, how='left', on=['product_id','application_id','loan_rate','loan_limit'])

In [34]:
a.rename(columns={'is_applied_y':'is_applied'}, inplace=True)
del a['is_applied_x']
a = a.fillna(0)
a = a[~a.duplicated()]

In [35]:
a = a[['application_id', 'product_id', 'is_applied']]

In [36]:
a = a[~a.duplicated(['application_id','product_id'])]

In [37]:
a

,application_id,product_id,is_applied
0,1748340,191,1.0
1,1748340,169,0.0
2,1748340,7,0.0
3,1748340,268,1.0
4,1748340,118,0.0
...,...,...,...
3257342,1428218,200,0.0
3257343,1428218,7,1.0
3257344,1428218,257,0.0
3257345,1428218,110,0.0


In [38]:
a.to_csv('데이터분석분야_퓨처스부문_수DA쟁이팀_평가데이터',index=False)